# Simple knapsack

## Intro

This is an integer linear programming problem in which the goal is to maximize the value transported in a knapsack limited by capacity by selecting the most suitable items.

## Model statement

$$
\begin{align}
    \text{max} \quad & \sum_{i \in I}{c_{i} x_{i}} \\
    \text{s.t.} \quad & \sum_{i \in I}{w_{i} x_{i}} \leq k \\
    & x_{i} \in \left \{ 0, 1 \right \} & \forall ~i \in I\\
\end{align}
$$

In [1]:
import numpy as np
from scipy.optimize import linprog
import pyomo.environ as pyo

In [2]:
# Set of items
I = list(range(1, 11))

# Random seed
np.random.seed(12)

# Weight associated with each item
w = dict(zip(I, np.random.normal(loc=5.0, scale=1.0, size=10).clip(0.5, 10.0)))

# Price associated with each item
price = dict(zip(I, np.random.normal(loc=10.0, scale=1.0, size=10).clip(0.5, 20.0)))

# knapsack capacity
k = 21.0

## Using pyomo

In [3]:
model = pyo.ConcreteModel()

### Sets

In [4]:
model.I = pyo.Set(initialize=I)

### Parameters

In [5]:
# Parameters of the knapsack
model.k = pyo.Param(initialize=k)

In [6]:
# Parameters of the items
model.w = pyo.Param(model.I, initialize=w)
model.c = pyo.Param(model.I, initialize=price)

### Variables

In [7]:
model.x = pyo.Var(model.I, within=pyo.Integers, bounds=(0, 1))

### Constraints

In [8]:
def capacity_constraint(model):
    return sum(model.x[i] * model.w[i] for i in model.I) <= model.k

model.capacity_constraint = pyo.Constraint(rule=capacity_constraint)

### Objective

In [9]:
def obj_function(model):
    return sum(model.x[i] * model.c[i] for i in model.I)
    
model.objective = pyo.Objective(rule=obj_function, sense=pyo.maximize)

### Solve

In [10]:
opt = pyo.SolverFactory('cbc')
opt.options['sec'] = 120

In [11]:
solution = opt.solve(model)

In [12]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 49.23378473504652


In [13]:
model.x.display()

x : Size=10, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   0.0 :     1 : False : False : Integers
      2 :     0 :   1.0 :     1 : False : False : Integers
      3 :     0 :   0.0 :     1 : False : False : Integers
      4 :     0 :   1.0 :     1 : False : False : Integers
      5 :     0 :   0.0 :     1 : False : False : Integers
      6 :     0 :   1.0 :     1 : False : False : Integers
      7 :     0 :   1.0 :     1 : False : False : Integers
      8 :     0 :   1.0 :     1 : False : False : Integers
      9 :     0 :   0.0 :     1 : False : False : Integers
     10 :     0 :   0.0 :     1 : False : False : Integers


## Using scipy

We must formulate this problem using the following matrix notation:

$$
\begin{align}
    \text{min} \quad & \boldsymbol{c}^T \boldsymbol{x} \\
    \text{s.t.} \quad & \boldsymbol{A}_{eq} \boldsymbol{x} = \boldsymbol{b}_{eq} \\
    & \boldsymbol{A}_{ub} \boldsymbol{x} \leq \boldsymbol{b}_{ub}\\
    & \boldsymbol{l} \leq \boldsymbol{x} \leq \boldsymbol{u}
\end{align}
$$

**Decision variables:**

$\boldsymbol{x}$ - Column vector with the amount of each item added to the knapsack

**Fixed parameters**

$\boldsymbol{c}$ - Negative value of the value of each item

$\boldsymbol{A}_{ub}$ - Row vector (two-dimensional in scipy) with the weight of each item $i$

$\boldsymbol{b}_{ub}$ - One-dimensional vector of knapsack capacity

In [14]:
# Costs
c = -np.array(list(price.values()))

# Inequality constraints matrix
A_ub = np.atleast_2d(list(w.values()))

# Upper bounds for linear inequality constraints
b_ub = np.array([k])

# Bounds (one quantity of each item)
bounds = [(0, 1),] * 10

In [15]:
sol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)
print(sol)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -50.63937756306515
              x: [ 0.000e+00  1.000e+00  1.371e-01  1.000e+00  0.000e+00
                   1.000e+00  1.000e+00  0.000e+00  1.000e+00  0.000e+00]
            nit: 1
          lower:  residual: [ 0.000e+00  1.000e+00  1.371e-01  1.000e+00
                              0.000e+00  1.000e+00  1.000e+00  0.000e+00
                              1.000e+00  0.000e+00]
                 marginals: [ 2.182e+00  0.000e+00  0.000e+00  0.000e+00
                              8.345e-01  0.000e+00  0.000e+00  1.242e+00
                              0.000e+00  5.451e+00]
          upper:  residual: [ 1.000e+00  0.000e+00  8.629e-01  0.000e+00
                              1.000e+00  0.000e+00  0.000e+00  1.000e+00
                              0.000e+00  1.000e+00]
                 marginals: [ 0.000e+00 -1.332e+00  0.000e+00 -1.802e+00
         

## Integrality constraints

Since version 1.9.0 scipy accepts integrality constraints as it now as a wrapper to a MILP solver.

$x_i \in \Z \quad \forall i \in I$

In [16]:
integrality_vector = np.full(c.shape[0], 1)

sol_int = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=integrality_vector)
print(sol_int)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -49.233784735046534
              x: [ 0.000e+00  1.000e+00 -0.000e+00  1.000e+00  0.000e+00
                   1.000e+00  1.000e+00  1.000e+00 -0.000e+00  0.000e+00]
            nit: -1
          lower:  residual: [ 0.000e+00  1.000e+00 -0.000e+00  1.000e+00
                              0.000e+00  1.000e+00  1.000e+00  1.000e+00
                             -0.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [ 1.000e+00 -8.882e-16  1.000e+00  0.000e+00
                              1.000e+00  0.000e+00  0.000e+00  0.000e+00
                              1.000e+00  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
       

In [17]:
for i, item in enumerate(I):
    if abs(sol.x[i] - model.x[item].value) <= 1e-3:
        if sol.x[i] >= 1e-3:
            print(f"Item {item} was added in both situations")
        else:
            print(f"Item {item} was not added in any situation")
        
    elif sol.x[i] > model.x[item].value + 1e-3:
        if sol.x[i] == 1:
            print(f"Item {item} was completely added in relaxed problem only")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem only - value {xi:.2f}")
            
    elif sol.x[i] + 1e-3 < model.x[item].value:
        if sol.x[i] <= 1e-3:
            print(f"Item {item} was added only in integer problem")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem - value {xi:.2f} - but completely added in the integer version")

Item 1 was not added in any situation
Item 2 was added in both situations
Item 3 was partially added in relaxed problem only - value 0.14
Item 4 was added in both situations
Item 5 was not added in any situation
Item 6 was added in both situations
Item 7 was added in both situations
Item 8 was added only in integer problem
Item 9 was completely added in relaxed problem only
Item 10 was not added in any situation


In [18]:
model.weight_constraint.display()

weight_constraint : Size=1
    Key  : Lower : Body              : Upper
    None :  None : 20.96801655429026 :  21.0
